In [1]:
import os
import numpy as np
import SimpleITK as sitk
from scipy.ndimage import center_of_mass, shift
import json

def resample_image(itk_image, out_spacing, is_label=False):
    original_spacing = itk_image.GetSpacing()
    original_size = itk_image.GetSize()
    original_direction = itk_image.GetDirection()
    original_origin = itk_image.GetOrigin()

    out_size = [
        int(round(osz * ospc / nspc))
        for osz, ospc, nspc in zip(itk_image.GetSize(), original_spacing, out_spacing)
    ]

    resampler = sitk.ResampleImageFilter()
    resampler.SetOutputSpacing(out_spacing)
    resampler.SetSize(out_size)
    resampler.SetOutputDirection(original_direction)
    resampler.SetOutputOrigin(original_origin)
    resampler.SetTransform(sitk.Transform())
    resampler.SetDefaultPixelValue(0)

    if is_label:
        resampler.SetInterpolator(sitk.sitkNearestNeighbor)
        # resampler.SetInterpolator(sitk.sitkLabelGaussian)
        # resampler.SetSigma(3.0)   # 控制边界模糊的程度，数值越大越模糊（默认1.0）
    else:
        resampler.SetInterpolator(sitk.sitkBSpline)

    return resampler.Execute(itk_image)


def normalize_image(image_np, percentile=(0.5, 99.5)):
    low, high = np.percentile(image_np, percentile)
    image_np = np.clip(image_np, low, high)
    image_np = (image_np - np.mean(image_np)) / np.std(image_np)
    return image_np

def save_nifti(array_np, reference_img, output_path):
    out_img = sitk.GetImageFromArray(array_np)
    out_img.SetSpacing(reference_img.GetSpacing())
    out_img.SetOrigin(reference_img.GetOrigin())
    out_img.SetDirection(reference_img.GetDirection())
    sitk.WriteImage(out_img, output_path)

def process_and_save(image_path, mask_path, save_dir, target_spacing=(1.0, 1.3, 1.3)):
    # Load image
    img_itk = sitk.ReadImage(image_path)
    mask_itk = sitk.ReadImage(mask_path)

    # Resample
    img_resampled = resample_image(img_itk, target_spacing, is_label=False)
    mask_resampled = resample_image(mask_itk, target_spacing, is_label=True)

    # Save resampled image as numpy array
    img_np = sitk.GetArrayFromImage(img_resampled)
    img_np = normalize_image(img_np)
    np.save(os.path.join(save_dir, "GED4.npy"), img_np)
    
    # Save resampled mask as numpy array
    mask_np = sitk.GetArrayFromImage(mask_resampled)
    np.save(os.path.join(save_dir, "mask_GED4.npy"), mask_np)

    # Save original affine
    metadata = {
        "original_spacing": img_itk.GetSpacing(),
        "original_shape": img_itk.GetSize(),
    }
    with open(os.path.join(save_dir, "metadata.json"), "w") as f:
        json.dump(metadata, f, indent = 4)
    
    # Save
    # os.makedirs(save_dir, exist_ok=True)
    # save_nifti(img_np, img_resampled, os.path.join(save_dir, "GED4_new.nii.gz"))

    # print(f"Saved processed image and mask to {save_dir}")

# === Example usage ===
# image_path = "/home/jincan/long_seg/my3DUNet/Data/MICCAI25/30labeled/1002-B1-S1/GED4.nii.gz"
# mask_path = "/home/jincan/long_seg/my3DUNet/Data/MICCAI25/30labeled/1002-B1-S1/mask_GED4.nii.gz"
# output_dir = "/home/jincan/long_seg/my3DUNet/Data/MICCAI25"

# process_and_save(image_path, mask_path, output_dir)


In [2]:
import os
import glob
import SimpleITK as sitk

data_dir = "./30labeled/train"
output_dir = "/home/lihang/Naive_3DUNet/Resampling_Data"
os.makedirs(output_dir, exist_ok=True)
output_dir = os.path.join(output_dir, "train")
os.makedirs(output_dir, exist_ok=True)
dir_list = os.listdir(data_dir)

for idx in range(len(dir_list)):
    image_path = os.path.join(data_dir, dir_list[idx], "GED4.nii.gz")
    mask_path = os.path.join(data_dir, dir_list[idx], "mask_GED4.nii.gz")
    
    save_dir = os.path.join(output_dir, dir_list[idx])
    os.makedirs(save_dir, exist_ok=True)

    process_and_save(image_path, mask_path, save_dir)

data_dir = "./30labeled/val"
output_dir = "/home/lihang/Naive_3DUNet/Resampling_Data"
output_dir = os.path.join(output_dir, "val")
os.makedirs(output_dir, exist_ok=True)
dir_list = os.listdir(data_dir)

for idx in range(len(dir_list)):
    image_path = os.path.join(data_dir, dir_list[idx], "GED4.nii.gz")
    mask_path = os.path.join(data_dir, dir_list[idx], "mask_GED4.nii.gz")
    
    save_dir = os.path.join(output_dir, dir_list[idx])
    os.makedirs(save_dir, exist_ok=True)

    process_and_save(image_path, mask_path, save_dir)


In [9]:
import numpy as np

Test_Path = "/home/lihang/Naive_3DUNet/Resampling_Data"
dir_list = os.listdir(Test_Path)
idx = 0
mask_original_path = os.path.join(Test_Path, dir_list[idx], "mask_GED4_original.nii.gz")
mask_resampled_path = os.path.join(Test_Path, dir_list[idx], "mask_GED4_resampled.nii.gz")
mask_resampled_back_path = os.path.join(Test_Path, dir_list[idx], "mask_GED4_resampled_back.nii.gz")

mask_original = sitk.ReadImage(mask_original_path)
mask_resampled = sitk.ReadImage(mask_resampled_path)
mask_resampled_back = sitk.ReadImage(mask_resampled_back_path)

# Convert to numpy arrays for visualization or further processing
mask_original_np = sitk.GetArrayFromImage(mask_original)
mask_resampled_np = sitk.GetArrayFromImage(mask_resampled)
mask_resampled_back_np = sitk.GetArrayFromImage(mask_resampled_back)

# Calculate the DICE bewtween the original and resampled back masks
def dice_coefficient(mask1, mask2):
    intersection = np.sum(mask1 * mask2)
    return 2.0 * intersection / (np.sum(mask1) + np.sum(mask2))
dice_original_resampled_back = dice_coefficient(mask_original_np, mask_resampled_back_np)
print(f"DICE coefficient between original and resampled back mask: {dice_original_resampled_back:.4f}")


DICE coefficient between original and resampled back mask: 0.9970
